In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

## La estructura de los mensajes

La forma en que funcionan estos modelos es a través de una interfaz de conversación, este motor de conversación funciona a través de una serie de mensajes en el formato:

```json
{
    "role": "user",
    "content": "message"
}
```

En cuanto a los roles, hay tres:

 - "user"
 - "assistant"
 - "system"

In [ ]:
messages = [
    { "role": "user", "content": "¿Quién eres?" }
]

In [ ]:
OPENAI_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0.7 # Value between 0 and 2

completions = client.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = TEMPERATURE,
    messages = messages,
)

In [ ]:
type(completions)

In [ ]:
message_choice = completions.choices[0]
message_choice

In [ ]:
message_choice.message.content

## El rol del `sistema`

Es posible utilizar un mensaje *"system"* para personalizar el asistente:

In [ ]:
messages = [
    {
        "role": "system", 
        "content": "Eres Tamal Enchilado, un famoso chef latinoamericano con experiencia en alta cocina"
    },
    { "role": "user", "content": "¿Quién eres?" }
]

In [ ]:
completions = client.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = TEMPERATURE,
    messages = messages,
)

In [ ]:
print(completions.choices[0].message.content)

## Una función auxiliar

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

def get_response(prompt, model="gpt-3.5-turbo", temperature=0.7):
    messages = [
        { "role": "user", "content": prompt }
    ]
    completions = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
    )
    return completions.choices[0].message.content
    

In [ ]:
response = get_response("Hola, mi nombre es Antonio, ¿quién eres tu?")

print(response)

## Resumir contenido

In [ ]:
three_star_review = """
No es un mal libro y es difícil discutir sobre lo que defienden los autores, \
pero para mí se parece mucho a un libro de autoayuda en el sentido de que la \
gente se sentirá bien mientras lo lee (y por tanto lo valorará muy positivamente), \
pero al final mucha gente no seguirá realmente el material y, a menos que seas un \
novato, gran parte de él será bastante obvio. Lo sé porque llevo 20 años trabajando \
en la industria del software y este libro se considera un clásico, pero la gente \
sigue cometiendo los mismos errores. \
En realidad, no es culpa del libro, pero también me da la impresión de que abarca \
mucho sin decir gran cosa. Es similar a un libro de autoayuda para personas que se \
sienten mal, puede que se sientan bien leyéndolo pero, ¿realmente te ayuda al final? \
Supongo que depende de la persona."""

### Primer intento

In [ ]:
prompt = "Resume el texto en una sola oración:"

In [ ]:
complete_prompt = prompt + " " + three_star_review
response = get_response(complete_prompt)

print(response)

### Segundo intento – usa delimitadores

In [ ]:
prompt = """Resume el texto delimitado por tres comillas invertidas en una sola oración:

```{text}```
"""

In [ ]:
complete_prompt = prompt.format(text=three_star_review)
print(complete_prompt)

In [ ]:
response = get_response(complete_prompt)

print(response)

### Tercer intento - pide una salida estructurada

In [ ]:
prompt = """Resume el texto delimitado por tres comillas invertidas en una sola oración.
Quiero la respuesta en formato JSON con las siguientes propiedades: summary (el resumen)

```{text}```
"""

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
import json

resp = json.loads(response)

resp['summary']

### Cuarto intento - verifica que está resumiendo reseñas de libros

In [ ]:
prompt = """Te voy a proporcionar un texto delimitado por tres comillas invertidas.\
Si el texto contiene una reseña de un libro, quiero que la resumas.
Si el texto no es una reseña de un libro, simplemente escribe \"No es reseña\".

Quiero la respuesta en formato JSON con las siguientes propiedades: summary (el resumen) y is_review (un booleano)

```{text}```
"""

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
not_a_review = """Fabricado con material de alta calidad, muy resistente y duradero. \
Pueden hacer que tu peinado sea más encantador y llamativo. \
Recibirás 2 lindas pinzas para el cabello, patas de pollo frito, pinzas para el cabello con alas de pollo. \
Mano de obra realista altamente simulada, como poner comida en tu cabeza, muy lindo. \
Regalo maravilloso: Las pinzas para el cabello creativas son un regalo bienvenido para amigos, \
familiares, esposas, novias, compañeros de clase y cualquier persona que ames. \
Fueron increíbles para la fiesta de cumpleaños temática de pollo frito."""

In [ ]:
response = get_response(prompt.format(text=not_a_review))

print(response)

### Fifth attempt - add more tasks

In [ ]:
prompt = """Te voy a proporcionar un texto delimitado por tres comillas invertidas.\
Si contiene una reseña sobre un libro, realice las siguientes acciones:

1. Identificar el sentimiento - sentiment
2. Extraer las palabras clave - keywords
3. Resumir la reseña - summary
4. Sugerir un título para la reseña - title

Si no contiene una reseña sobre un libro, simplemente escriba \"No es reseña\"

Proporcione la respuesta como un objeto JSON con las siguientes propiedades: summary, is_a_review, keywords, title, sentiment

```{text}```
"""

In [ ]:
response = get_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
response = get_response(prompt.format(text=not_a_review))

print(response)

### Sexto intento – "fuerza" al modelo para que regrese JSON

#### Solo disponible para `gpt-4-1106-preview`!

In [ ]:
def get_json_response(prompt, temperature = 0.7):
    messages = [
        { "role": "user", "content": prompt }
    ]
    completions = client.chat.completions.create(
        model = "gpt-4-1106-preview",
        temperature = temperature,
        messages = messages,
        
        response_format={ "type": "json_object" }

    )
    return completions.choices[0].message.content
    

In [ ]:
response = get_json_response(prompt.format(text=three_star_review))

print(response)

In [ ]:
resp = json.loads(response)

print("""Título de la reseña: {title}
Palabras clave: {keywords}
sentimiento: {sentiment}
""".format(**resp))

In [ ]:
response = get_json_response("¿Quién eres?")

print(response)

## Expansión de contenido

In [ ]:
prompt = """Voy a darte las especificaciones de un {product}.
Las especificaciones están delimitadas por corchetes angulares.
Quiero que escribas una breve descripción del mismo utilizando las especificaciones pero destacando su {highlight}.

>>>
{specs}
>>>
"""

In [ ]:
full_prompt = prompt.format(
    product='Consola de videojuegos',
    highlight='portabilidad',
    specs="""Tamaño: Aproximadamente 4 pulgadas (10.16 cm) de alto, 9.5 pulgadas (24.13 cm) de ancho y 0.55 pulgadas (1.4 cm) de largo (con los controles acoplados).
Peso: Aproximadamente 0.71 libras (322 g), 0.93 libras (426.38 g) con los controles acoplados
CPU/GPU: Procesador NVIDIA Custom Tegra
Almacenamiento: 64 GB (Los usuarios pueden ampliar la capacidad de almacenamiento con tarjetas microSDHC o microSDXC hasta de 2 TB).
Conexión inalámbrica: Wi-Fi (cumple con IEEE 802.11 a/b/g/n/ac), Bluetooth 4.1
Salida de video: 1080p via HDMI y 720p via la pantalla incluída.
Salida de audio: Altavoces estéreo
Botones: Botón POWER / Botón de volumen
Conector USB: Conector USB Type-C Se utiliza para cargar la batería o para conectarse con la base de Nintendo Switch.
Conector de audio: Conector de 3.5 mm con 4 polos (estándar para CTIA)
""")
    
print(full_prompt)

In [ ]:
product_description = get_response(full_prompt)

print(product_description)

In [ ]:
full_prompt = prompt.format(
    product='Lavadora automática',
    highlight='olor',
    specs="""Tamaño: Aproximadamente 4 pulgadas (10.16 cm) de alto, 9.5 pulgadas (24.13 cm) de ancho y 0.55 pulgadas (1.4 cm) de largo (con los controles acoplados).
Peso: Aproximadamente 0.71 libras (322 g), 0.93 libras (426.38 g) con los controles acoplados
CPU/GPU: Procesador NVIDIA Custom Tegra
Almacenamiento: 64 GB (Los usuarios pueden ampliar la capacidad de almacenamiento con tarjetas microSDHC o microSDXC hasta de 2 TB).
Conexión inalámbrica: Wi-Fi (cumple con IEEE 802.11 a/b/g/n/ac), Bluetooth 4.1
Salida de video: 1080p via HDMI y 720p via la pantalla incluída.
Salida de audio: Altavoces estéreo
Botones: Botón POWER / Botón de volumen
Conector USB: Conector USB Type-C Se utiliza para cargar la batería o para conectarse con la base de Nintendo Switch.
Conector de audio: Conector de 3.5 mm con 4 polos (estándar para CTIA)
""")

product_description = get_response(full_prompt)

print(product_description)

## Aprendizaje "few-shot"

Vamos a inventar [nuevos nombres](https://en.wikipedia.org/wiki/Boaty_McBoatface) "británicos".

In [ ]:
prompt = """Te voy a dar un un sustantivo delimitado por corchetes angulares, \
quiero que me des un nombre para una un objeto de ese sustantivo.

>>>
{noun}
>>>
"""

In [ ]:
response = get_response(prompt.format(noun='Egg'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='Train'))
print(response)

In [ ]:
prompt = """Te voy a dar un un sustantivo delimitado por corchetes angulares, \
quiero que me des un nombre para una un objeto de ese sustantivo.

User: boat
Assistant: Boaty McBoatface

User: howitzer
Assistant: Cannon McCannonface

User: beer
Assistant: Lager McLagerface

>>>
{noun}
>>>
"""

In [ ]:
response = get_response(prompt.format(noun='Egg'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='Train'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='snow plow'))
print(response)

In [ ]:
response = get_response(prompt.format(noun='guimpe'))
print(response)

## ¿Tokens?

Aquí está la documentación: https://platform.openai.com/tokenizer

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model(OPENAI_MODEL)

In [ ]:
tokens = encoding.encode("Hola, ¿cómo estás?")
print(tokens)

In [ ]:
decoded = encoding.decode(tokens)
print(decoded)

In [ ]:
encoding.decode_single_token_bytes(69112)

In [ ]:
encoding.decode_single_token_bytes(69113)

In [ ]:
encoding.decode_single_token_bytes(69111)